## Azure Quantum Q# notebook

To use Jupyter Notebooks with Q#, the "qsharp" and "jupyterlab" Python packages should be installed.

To render charts as shown in this notebook, Matplotlib with notebook widgets support should be installed via "ipympl".

To submit to Azure Quantum as shown in this notebook, the "azure-quantum" package needs to be installed.

You can install all the above packages in your Python environment by running the below in your terminal:

```bash
pip install jupyterlab qsharp ipympl azure-quantum
```

In [ ]:
# Run this cell first to enable the "%%qsharp" magic command used in later cells
import qsharp


The below cell shows Q# being written directly inside a Jupyter Notebook using the `%%qsharp` 'magic' command

In [ ]:
%%qsharp

// This makes the DumpMachine() function available.
import Std.Diagnostics.*;

operation RandomBit() : Result {
    // Qubits are only accesible for the duration of the scope where they
    // are allocated and are automatically released at the end of the scope.
    use qubit = Qubit();

    // Set the qubit in superposition by applying a Hadamard transformation.
    H(qubit);

    DumpMachine();

    // Measure the qubit. There is a 50% probability of measuring either
    // "Zero" or "One".
    let result = M(qubit);

    Message($"The result of the measurement is {result}.");

    // Reset the qubit so it can be safely released.
    Reset(qubit);
    return result;
}

// Call the operation we defined above.
RandomBit();


As well as invoking Q# operations immediately inline (as the last line in the cell above does), the defined operations can also be invoked from Python

In [ ]:
r = qsharp.eval("RandomBit()")

if r == qsharp.Result.One:
    i = 1
else:
    i = 0

print(f"Python: Got integer {i}!")


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Use save_events to suppress the automatic printing of events for every shot
results = qsharp.run("RandomBit()", shots=1000, save_events=True)

# Sort the results so that the histogram labels appear in the correct order
# We only care about the result values, not the event data, so extract just the result
results = [entry["result"] for entry in results]
results.sort()

# Count the number of times each result appears
counts = Counter(results)

(values, counts) = counts.keys(), counts.values()
xlabels = np.arange(len(counts))
plt.title("RandomBit() Results")
plt.bar(xlabels, counts)
plt.xticks(xlabels, values)
plt.show()


## Q# widgets

You can also use the `qsharp_widgets` package to visualize data. Install with `pip install qsharp-widgets`

In [ ]:
from qsharp_widgets import Histogram

Histogram(results)


In [ ]:
from qsharp_widgets import EstimatesPanel

estimate = qsharp.estimate("RandomBit()", [
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_ns_e3"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_us_e4"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_maj_ns_e6"}, "qecScheme": {"name": "floquet_code"}}
])
EstimatesPanel(estimate)


## Submitting jobs to Azure Quantum

Different quantum hardware supports different capabilities, but all Azure Quantum providers support the 'base profile'
as defined in the 'Quantum Intermediate Representation' (QIR) specification. (For more details see <https://aka.ms/qdk.qir>)

To develop code using this base profile, reintialize the Q# compiler, connect to your Azure Quantum workspace, and submit the job.


In [ ]:
# Reset the compiler to target the 'base profile' that all hardware supports

qsharp.init(target_profile=qsharp.TargetProfile.Base)


In [ ]:
# Connect to the Azure Quantum workspace

# If developing locally, on first run this will open a browser to authenticate the
# connection with Azure. In remote scenarios, such as SSH or Codespaces, it may
# be necessary to install the Azure CLI and run 'az login --use-device-code' to
# authenticate. For unattended scenarios, such as batch jobs, a service principal
# should be configured and used for authentication. For more information, see
# https://learn.microsoft.com/en-us/azure/developer/python/sdk/authentication-overview

from azure.quantum import Workspace

# If using an access key, replace the below with: Workspace.from_connection_string(connection_string)
# Or set the "AZURE_QUANTUM_CONNECTION_STRING" environment variable and just use: Workspace()
# See https://learn.microsoft.com/en-us/azure/quantum/how-to-connect-workspace for more details.

workspace = Workspace(
    subscription_id = "MY_SUBSCRIPTION_ID",
    resource_group = "MY_RESOURCE_GROUP",
    name = "MY_WORKSPACE_NAME",
    location = "MY_WORKSPACE_LOCATION",
)


In [ ]:
%%qsharp

operation Random() : Result {
    use q = Qubit();
    H(q);
    let result = M(q);
    Reset(q);
    return result
}

operation RandomNBits(N: Int): Result[] {
    mutable results = [];
    for i in 0 .. N - 1 {
        let r = Random();
        set results += [r];
    }
    return results
}


In [ ]:
# Run the above code for 100 shots against the Rigetti simulator

operation = qsharp.compile("RandomNBits(4)")
target = workspace.get_targets("rigetti.sim.qvm")
job = target.submit(operation, "my-azure-quantum-job", shots=100)

# Wait for the job to complete
job.get_results()
